<a href="https://colab.research.google.com/github/rahiakela/computer-vision-research-and-practice/blob/main/06-image-preprocessing/practical-machine-learning-for-computer-vision/01_image_size_and_resolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Image Size and Resolution

In the standard image processing pipeline, The next
stage is preprocessing the raw images in order to feed them into the model for training or inference.

![](https://github.com/rahiakela/computer-vision-research-and-practice/blob/main/06-image-preprocessing/practical-machine-learning-for-computer-vision/images/training_pipeline.png?raw=1)

Before raw images can be fed into an image model, they usually have to be preprocessed.

Such preprocessing has several overlapping goals: shape transformation, data
quality, and model quality.

1. The input images typically have to be transformed into a consistent size.
2. Another reason to do preprocessing is to enforce data quality.
3. A third goal of preprocessing is to carry out transformations that help improve the accuracy of models trained on the data.

##Setup

In [ ]:
import tensorflow as tf

import matplotlib.pylab as plt
import numpy as np

In [ ]:
# Helper functions
def training_plot(metrics, history):
  f, ax = plt.subplots(1, len(metrics), figsize=(5*len(metrics), 5))
  for idx, metric in enumerate(metrics):
    ax[idx].plot(history.history[metric], ls='dashed')
    ax[idx].set_xlabel("Epochs")
    ax[idx].set_ylabel(metric)
    ax[idx].plot(history.history['val_' + metric]);
    ax[idx].legend([metric, 'val_' + metric])

In [ ]:
def parse_tfr(proto):
  feature_description = {
      'image': tf.io.VarLenFeature(tf.float32),
      'shape': tf.io.VarLenFeature(tf.int64),
      'label': tf.io.FixedLenFeature([], tf.string, default_value=''),
      'label_int': tf.io.FixedLenFeature([], tf.int64, default_value=0),
  }
  rec = tf.io.parse_single_example(
      proto, feature_description
  )
  shape = tf.sparse.to_dense(rec['shape'])
  img = tf.reshape(tf.sparse.to_dense(rec['image']), shape)
  label_int = rec['label_int']
  return img, label_int

In [ ]:
IMG_HEIGHT = 224
IMG_WIDTH = 224
IMG_CHANNELS = 3
CLASS_NAMES = 'daisy dandelion roses sunflowers tulips'.split()

##Loading Images